In [25]:
import pandas as pd
df = pd.read_excel('poblacion_suelo.xlsx')

In [26]:
import geopandas as gpd
from shapely import wkt

df['geometry'] = df['geometry'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

print(type(gdf))
gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,Unnamed: 0,geometry,lat,lon,POBTOT,POBFEM,POBMAS,POB0_14,P15A29A,P30A59A,...,LETRERO_C,TELPUB_C,ARBOLES_C,DRENAJEP_C,TRANSCOL_C,ACESOPER_C,ACESOAUT_C,PUESSEMI_C,PUESAMBU_C,SUELO
0,0,"POLYGON ((-100.34877 25.75497, -100.34889 25.7...",25.755132,-100.348732,31,17,14,10,11,8,...,3,3,2,1,3,3,3,3,3,residential
1,1,"POLYGON ((-100.40637 25.76005, -100.40642 25.7...",25.760381,-100.406710,59,31,28,14,9,32,...,1,3,2,3,3,2,2,3,3,residential
2,2,"POLYGON ((-100.35263 25.7418, -100.35405 25.74...",25.742596,-100.353221,183,97,86,30,36,68,...,2,2,2,2,2,3,3,3,3,residential
3,3,"POLYGON ((-100.40797 25.75579, -100.40803 25.7...",25.756088,-100.408082,26,11,15,7,-1,17,...,2,3,2,3,3,1,1,3,3,residential
4,4,"POLYGON ((-100.41435 25.75627, -100.4152 25.75...",25.756496,-100.414803,67,36,31,19,13,34,...,3,3,3,3,3,3,3,3,3,residential


In [33]:
import numpy as np
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
from typing import List, Tuple, Optional, Literal

# ---------- Parámetros de tu modelo ----------
BASE_COLS = [
    'GRAPROES','GRAPROES_F','GRAPROES_M','RECUCALL_C','RAMPAS_C','PASOPEAT_C',
    'BANQUETA_C','GUARNICI_C','CICLOVIA_C','CICLOCAR_C','ALUMPUB_C','LETRERO_C',
    'TELPUB_C','ARBOLES_C','DRENAJEP_C','TRANSCOL_C','ACESOPER_C','ACESOAUT_C',

]

# ---------- Helpers de geometría ----------
import numpy as np, pandas as pd, geopandas as gpd
from shapely.geometry import Point
from typing import List, Tuple, Optional, Literal

def to_geodf(df_or_gdf, lat_col="lat", lon_col="lon", crs="EPSG:4326") -> gpd.GeoDataFrame:
    """
    Acepta un DataFrame (con lat/lon) o un GeoDataFrame y devuelve un GeoDataFrame con geometría y CRS.
    """
    if isinstance(df_or_gdf, gpd.GeoDataFrame):
        gdf = df_or_gdf.copy()
        if gdf.crs is None:
            gdf.set_crs(crs, inplace=True)
        return gdf
    # Es un DataFrame: necesita lat/lon
    df = df_or_gdf.copy()
    if lat_col not in df.columns or lon_col not in df.columns:
        raise ValueError(f"Faltan columnas '{lat_col}' y/o '{lon_col}' para construir geometría.")
    # valida rangos
    bad = (~df[lat_col].between(-90, 90)) | (~df[lon_col].between(-180, 180))
    if bad.any():
        idx_bad = df.index[bad].tolist()[:5]
        raise ValueError(f"Hay lat/lon fuera de rango en filas {idx_bad} (muestra).")
    geom = gpd.GeoSeries([Point(lon, lat) for lat, lon in zip(df[lat_col], df[lon_col])], crs=crs)
    gdf = gpd.GeoDataFrame(df, geometry=geom, crs=crs)
    return gdf

def _ensure_crs(gdf: gpd.GeoDataFrame, default="EPSG:4326") -> gpd.GeoDataFrame:
    gdf = gdf.copy()
    if gdf.crs is None:
        gdf.set_crs(default, inplace=True)
    return gdf

def _utm_guess(gdf_4326: gpd.GeoDataFrame) -> str:
    """Elige automáticamente una zona UTM según el centroide (para buffers/áreas en metros)."""
    c = gdf_4326.unary_union.envelope.centroid
    lon, lat = float(c.x), float(c.y)
    zone = int((lon + 180) // 6) + 1
    epsg = 32600 + zone if lat >= 0 else 32700 + zone
    return f"EPSG:{epsg}"

def make_zone_from_points(
    coords_latlon: List[Tuple[float, float]],
    method: Literal["hull","buffer"] = "hull",
    buffer_m: float = 150
) -> gpd.GeoDataFrame:
    """
    Crea una zona poligonal a partir de puntos (lat,lon).
    - 'hull': convex hull que envuelve todos los puntos
    - 'buffer': buffer (en metros) de cada punto y unión
    """
    if len(coords_latlon) == 0:
        raise ValueError("Debes pasar al menos un (lat, lon).")
    # puntos en WGS84
    pts = gpd.GeoSeries([Point(lon, lat) for lat, lon in coords_latlon], crs="EPSG:4326")
    # Para buffer en metros: reproyecta a UTM adecuada
    utm = _utm_guess(gpd.GeoDataFrame(geometry=pts))
    pts_utm = pts.to_crs(utm)
    if method == "hull":
        poly_utm = pts_utm.unary_union.convex_hull
    elif method == "buffer":
        poly_utm = unary_union([p.buffer(buffer_m) for p in pts_utm])
    else:
        raise ValueError("method debe ser 'hull' o 'buffer'")
    poly_wgs = gpd.GeoSeries([poly_utm], crs=utm).to_crs("EPSG:4326")
    return gpd.GeoDataFrame({"zone_id":[0]}, geometry=poly_wgs)

# ---------- Agregación dentro de la zona ----------
def aggregate_zone_variables(
    gdf_f: gpd.GeoDataFrame,
    zone_gdf: gpd.GeoDataFrame,
    cols: List[str] = BASE_COLS,
    agg: Literal["median","mean","pw_mean"] = "median",
    weight_col: Optional[str] = None
) -> pd.DataFrame:
    """
    Interseca la zona con gdf_f y agrega columnas numéricas:
    - median / mean: estadístico simple
    - pw_mean: promedio ponderado por 'weight_col' (p. ej., POBTOT)
    Retorna un DataFrame con UNA fila (la zona).
    """
    gdf_f = _ensure_crs(gdf_f)
    zone_gdf = _ensure_crs(zone_gdf)
    # trabajar en 4326 para rapidez (no calculamos áreas aquí)
    if gdf_f.crs.to_string() != zone_gdf.crs.to_string():
        zone_gdf = zone_gdf.to_crs(gdf_f.crs)

    # intersección espacial (zonas de gdf_f que caen dentro de la zona ad-hoc)
    # si gdf_f son polígonos: within/intersects; si son puntos: within/contains
    try:
        sub = gpd.sjoin(gdf_f, zone_gdf[["geometry"]], predicate="intersects", how="inner")
    except Exception:
        sub = gpd.sjoin(gdf_f, zone_gdf[["geometry"]], predicate="within", how="inner")

    if sub.empty:
        raise ValueError("La zona no intersecta con ninguna fila de gdf_f.")

    present = [c for c in cols if c in sub.columns]
    if not present:
        raise ValueError("Ninguna columna de BASE_COLS está en gdf_f para agregar.")

    # casteo seguro
    sub[present] = sub[present].apply(pd.to_numeric, errors="coerce")

    if agg == "median":
        row = sub[present].median(numeric_only=True).to_frame().T
    elif agg == "mean":
        row = sub[present].mean(numeric_only=True).to_frame().T
    elif agg == "pw_mean":
        if not weight_col or weight_col not in sub.columns:
            raise ValueError("Para 'pw_mean' debes indicar weight_col presente en gdf_f (p.ej., 'POBTOT').")
        w = pd.to_numeric(sub[weight_col], errors="coerce").fillna(0).values.reshape(-1,1)
        X = sub[present].fillna(sub[present].median()).values
        wsum = np.sum(w) if np.sum(w) > 0 else 1.0
        pwm = (w.T @ X) / wsum
        row = pd.DataFrame(pwm, columns=present)
    else:
        raise ValueError("agg debe ser 'median', 'mean' o 'pw_mean'.")

    row.reset_index(drop=True, inplace=True)
    row["n_obs_in_zone"] = len(sub)
    return row

# ---------- Pipeline: puntos → zona → agregación → recomendación ----------
def recommend_for_zone_points(
    recommender,             # instancia ya entrenada de PCARecommender
    gdf_f: gpd.GeoDataFrame,
    coords_latlon: List[Tuple[float, float]],
    method: Literal["hull","buffer"] = "hull",
    buffer_m: float = 150,
    agg: Literal["median","mean","pw_mean"] = "median",
    weight_col: Optional[str] = None
) -> Tuple[pd.DataFrame, gpd.GeoDataFrame]:
    """
    Devuelve (recomendacion_df_con_una_fila, zone_polygon_gdf)
    """
    zone = make_zone_from_points(coords_latlon, method=method, buffer_m=buffer_m)
    zone_agg = aggregate_zone_variables(gdf_f, zone, cols=recommender.cols, agg=agg, weight_col=weight_col)
    res = recommender.transform(zone_agg)
    # empaqueta con metadata útil
    out = res["recommendations"].copy()
    out["aggregation"] = agg
    out["method"] = method
    out["n_obs_in_zone"] = zone_agg["n_obs_in_zone"].iloc[0]
    return out, zone


In [31]:
import importlib
import pca_recommender     # o import app.pca_recommender si está en un paquete

importlib.reload(pca_recommender)   # <- recarga el módulo
from pca_recommender import PCARecommender  # <- reimporta los símbolos


In [34]:
from pca_recommender import PCARecommender
rec = PCARecommender(cols=BASE_COLS, var_target=0.80, top_k_loadings=5)
rec.fit(df)  

coords = [
    (25.755132, -100.348732),
    (25.760381, -100.406710),
    (25.742596, -100.353221), 
]

gdf_base = to_geodf(df, lat_col="lat", lon_col="lon")  
rec_df, zone_poly = recommend_for_zone_points(
    recommender=rec,
    gdf_f=gdf_base,           
    coords_latlon=coords,
    method="buffer",
    buffer_m=200,
    agg="pw_mean",
    weight_col="POBTOT"        
)

print(rec_df[["weak_component","worst_feature","recommended_intervention","n_obs_in_zone"]])



  weak_component worst_feature             recommended_intervention  \
0            PC1    BANQUETA_C  Sidewalks & walkability (banquetas)   

   n_obs_in_zone  
0             30  


/var/folders/l9/hx3k3fcs03d6gz71m2n3t1c80000gn/T/ipykernel_33490/1556788581.py:49: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  c = gdf_4326.unary_union.envelope.centroid
